In [30]:
def write(keyPath, obj={}, writeVal=''):
  paths = [comp for comp in keyPath.split('/') if len(comp) > 0]
  key = paths[0] if len(paths) > 0 else None
  rest = '/'.join(paths[1:]) if len(paths) > 1 else None
  if not rest:
    if key == '[]':
      for i in range(0, len(obj)):
        obj[i] = writeVal
    elif key[0] == '[':
      index = int(key[1:-1])
      obj[index] = writeVal
    else:
      obj[key] = writeVal
  else:
    if key == '[]':
      for element in obj:
        write(rest, obj=element, writeVal=writeVal)
    elif key[0] == '[':
      index = int(key[1:-1])
      write(rest, obj=obj[index], writeVal=writeVal)
    else:
      if key not in obj:
        obj[key] = {}
      write(rest, obj=obj[key], writeVal=writeVal)

In [40]:
def find(keyPath, obj={}, do=lambda x:x, cast=None, writeVal=None):
  if writeVal:
    return write(keyPath, obj=obj, writeVal=writeVal)
  paths = [comp for comp in keyPath.split('/') if len(comp) > 0]
  key = paths[0] if len(paths) > 0 else None
  rest = '/'.join(paths[1:])
  if not key:
    return cast(do(obj)) if cast else do(obj)
  elif key == '[]':
    return [find(rest, obj=value, do=do, cast=cast) for value in obj]
  elif key[0] == '[':
    index = int(key[1:-1])
    return find(rest, obj=obj[index], do=do, cast=cast) if len(obj) > index else None        
  else:
    if key in obj:
      return find(rest, obj=obj[key], do=do, cast=cast)

In [41]:
import os, datetime
def finalWrite(dictionary, label, date=datetime.datetime.now().strftime("%b %-d %Y %-I.%M %p"), cmdline=False):
    pathname = 'Final/' + date + '/' + label + '.json'
    if not os.path.exists(os.path.dirname(pathname)):
        try:
            os.makedirs(os.path.dirname(pathname))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    with open(pathname, 'w') as file:
        file.write(json.dumps(dictionary, indent=2))
    return pathname.replace(' ', '\ ') if cmdline else pathname

In [39]:
def copyKeys(root, tupleOrKeys):
    copy = {}
    for tupOrKey in tupleOrKeys:
      if isinstance(tupOrKey, tuple):
        key, path = tupOrKey
      else:
        key, path = None, tupOrKey
      key = key if key else path
      value = find(path, obj=root)
      copy.update({key: value} if value else {})
    return copy

In [19]:
class pluck():
  def __init__(self, *args, **kwargs):
    self.args = args
  def getTuple(self, dictionary):
    vals = [dictionary[arg] if arg in dictionary else None for arg in self.args]
    return tuple(vals) if len(vals) > 1 else vals[0]
  def all(self, lst):
    return [self.getTuple(x) for x in lst]

In [18]:
def name(fullName, match=False):
  if match:
    names = fullName.replace('.', '').lower().split(' ')
    return names[0] + ' ' + names[-1]
  else:
    names = [nm for nm in fullName.split(' ') if '.' not in nm and len(nm) != 1]
    if len(names) == 1:
      names = fullName.split(' ')
    return names[0] + ' ' + ' '.join(names[1:])

In [17]:
def combineDicts(dicts):
  def mergeDicts(a, b):
    a.update(b)
    return a
  return reduce(mergeDicts, dicts, {})

In [5]:
allWith = lambda keyPath, lst, do=lambda x:x, cast=None: [(x, val) for x, val in [(x, find(keyPath, obj=x, do=do, cast=cast)) for x in lst] if val]

In [13]:
from collections import Counter

class Mapper():
  
  @staticmethod
  def noChange(toKey):
    return lambda value: {toKey: value} if value else {}
  
  def __init__(self, fromKeys, fn):
    self.fromKeys = fromKeys
    self.fn = fn

  def map_samples(self, samples):
    for i, sample in samples:
      self.map(sample, sampleIndex=i)
  
  def run(self, rawValues):
    for raw in rawValues:
      self.map(raw)
  
  def uniques(self, rawValues, start, end, keyPath=None):
    counter = Counter()
    def countIt(v):
      counter[v] += 1
    for raw in rawValues:
      output = {}
      self.map(raw, to=output)
      if len(output.values()) == 0:
        countIt('Not Found')
      elif keyPath:
        find(keyPath, obj=output, do=countIt)
      else:
        for val in output.values():
          countIt(val)
    sorted = [tup[0] for tup in counter.most_common()][start:end]
    for value in sorted:
      print value
    
  def map(self, raw, to={}, sampleIndex=-1):
    if isinstance(self.fromKeys, list):
      inputs = tuple(raw[key] if key in raw else None for key in self.fromKeys)
    elif self.fromKeys not in raw:
      if sampleIndex != -1:
        print self.fromKeys, 'not found in', sampleIndex
      return
    else:
      inputs = raw[self.fromKeys]
    outputs = self.fn(inputs)
    if sampleIndex != -1:
      print 'Input', str(sampleIndex) + ':'
      print json.dumps(inputs, indent=2)
      print 'into output:'
      print json.dumps(outputs, indent=2)
    else:
      to.update(outputs)

In [16]:
from random import randint
def genSample(items):
  return items[randint(0, len(items))]
def genSamples(items, n, includeIndices=True):
  count = len(items)
  indices = [randint(0, count) for _ in range(0, n)]
  if includeIndices:
    return [(i, items[i]) for i in indices]
  else:
    return [items[i] for i in indices]

In [15]:
import re
class MultiReplace():  
  def __init__(self, replacements):
    self.replacements = [(re.compile(regex), replaceWith) for regex, replaceWith in replacements.items()]

  def sub(self, string):
    for regex, replaceWith in self.replacements:
      string = regex.sub(replaceWith, string)
    return string

In [1]:
def indexOfClosest(needle, haystack):
  N = len(haystack); floor = 0; ceil = N;
  if N == 0:
    return None
  k = N / 2
  while ceil - floor != 1 and haystack[k] != needle:
    if haystack[k] > needle:
      ceil = k
    elif haystack[k] < needle:
      floor = k
    k = (ceil - floor) / 2 + floor
  return k
  
def percentileOfValue(needle, haystack):
  N = len(haystack)
  if N == 0:
    return None
  k = indexOfClosest(needle, haystack)
  if haystack[k] == needle:
    floor = ceil = k
    while floor > -1 and haystack[floor] == needle:
      floor -= 1
    while ceil < N and haystack[ceil] == needle:
      ceil += 1
    index = float(ceil + floor) / 2.0
  else:
    index = float(k)
  return int(100 * (index / float(N)))

In [5]:
def analyze(key, objects):
  filtered = [o for o in objects if key in o]
  print 'Is optional?', len(filtered) != len(objects), len(filtered), 'of', len(objects)
  from collections import Counter
  import json
  valCounter = Counter()
  typeCounter = Counter()
  for o in filtered:
    value = o[key] if type(o[key]) == unicode else json.dumps(o[key])
    valCounter[value] += 1
    typeCounter[type(o[key])] += 1
  print typeCounter
  print valCounter.most_common(10)